In [15]:
import requests
import pandas as pd
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy
import time

In [2]:
def get_conn_readreplica(SSH_required,key_path):   #for getting a connection as a result

    db='datawarehouse'
    DB_HOST='gs-prod-new-db-read.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [6]:
def get_df_from_sql_readreplica(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='gs-prod-new-db-read.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [22]:
def make_request(address, city):
    url = 'http://grow-simplee-nlb-prod-a264c46571856f67.elb.ap-south-1.amazonaws.com:9016/parse_address'
    headers = {'Content-Type': 'application/json'}
    data = {
        "address": address,
        "city": city
    }
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # Raise an error for bad status codes
        
        response_json = response.json()
        
        # Extract lat_lng from the response JSON
        lat_lng = response_json.get('geo_entities', {}).get('lat_lng', None)
        return lat_lng if lat_lng else None
        
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        print(f"Response content: {response.content}")
        return None

In [10]:
# Usage with the actual path to the private key
SSH_required = 'Yes'
key_path = '/home/rajat/Downloads/tunnel-ssh .cer'
query = "select * from public.warehouses_warehouse"

# Establish a connection
conn = get_conn_readreplica(SSH_required, key_path)

# Retrieve data into a DataFrame
df_warehouse = get_df_from_sql_readreplica(SSH_required, query, key_path)

# Now you can perform further operations with the DataFrame 'df_warehouse'
print(df_warehouse)

Tunnel Started
Connection Made
      warehouse_int_id                                       warehouse_id  \
0                 1109                                       Aditya Gupta   
1                 1461                         Harihar Bhiwandi Warehouse   
2                 1471                       S0002 Bagru Hastkala Printer   
3                 1467                                             Source   
4                   31                                RHJ(Jagraon_Punjab)   
...                ...                                                ...   
1938              2059     16   1, Deganhalli Village Road, Kasaba Hobli,   
1939              2060  Khasra No  121   14 15 16 1 7 2 122   10 2 11 ...   
1940              2061  Khasra No  121   14 15 16 1 7 2 122   10 2 11 ...   
1941              2062  Honasa Consumer Limited  Khasra Nos  205  25 2...   
1942              2069                                  GP 50 Udyog Vihar   

                                         war

In [ ]:
lat_long_list = []

# Iterate through the DataFrame row by row
for index, row in df_warehouse.iterrows():
    lat_long = make_request(row['warehouse_address'], row['warehouse_city'])
    lat_long_list.append(lat_long)
    time.sleep(5)  # Delay between requests to avoid overloading the server

# Add the results to the DataFrame
df_warehouse['lat_long'] = lat_long_list

# Print the DataFrame with latitude and longitude
print("\nDataFrame with lat_long:")
print(df_warehouse)

An error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: http://grow-simplee-nlb-prod-a264c46571856f67.elb.ap-south-1.amazonaws.com:9016/parse_address
Response content: b'<!doctype html>\n<html lang=en>\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'
An error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: http://grow-simplee-nlb-prod-a264c46571856f67.elb.ap-south-1.amazonaws.com:9016/parse_address
Response content: b'<!doctype html>\n<html lang=en>\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'
An error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: http://grow-simplee-nlb-